In [5]:
import pandas
import numpy
from numpy import std, correlate, sqrt
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels.stats.mediation import Mediation
from linearmodels import IV2SLS
%run Table_Summary.ipynb
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
plt.style.use('seaborn-whitegrid')
from math import pi
import openpyxl

# Data

In [6]:
filepath = 'G:/My Drive/Elio/Attention/Data/Original_data.txt'
data = pandas.read_csv(filepath,sep='\t')
data.rename(columns={'origi':'Id'},inplace=True)
print(data.shape,'from 2019_05_25_Results.xlxs')
print(data.Year.unique(),'Years')
print(data.Id.nunique(),'Number of firms')
filepath = 'G:/My Drive/Elio/Attention/Data/Database_secondreviewJIBS.dta'
data1 = pandas.read_stata(filepath)
data1.rename(columns={'id':'Id','year':'Year','country':'Country'},inplace=True)
data1 = data1[['Id','Year','Country']]
data1['Year'] = pandas.to_numeric(data1.Year.astype(str).str[0:4])
print(data1.shape,'to get country')
data = pandas.merge(left=data,right=data1,how='left',on=['Year','Id'])
print(data.Country.unique())
filepath = 'G:/My Drive/Elio/Attention/Data/2019_05_25_Results_updated.txt'
data2 = pandas.read_csv(filepath,sep='\t')
print(data2.shape,'correction by Elio of market commonality')
data2.rename(columns={'origi':'Id'},inplace=True)
data2 = data2[['Id','Year','Market_commonality']]
data.drop('Market_commonality',axis=1,inplace = True)
data = pandas.merge(left=data,right=data2,how='inner',on=['Year','Id'])
print(data.Non_RD_alliances.sum(),'Total number of Non-RD alliances')
print(data.RD_alliances.sum(),'Total number of RD alliances (hybrid alliances)')
filepath = 'G:/My Drive/Elio/Attention/Data/Count/Data/count_sim.txt'
data3 = pandas.read_csv(filepath,sep='\t')
data3['SUM'] = data3['geographic']+data3['growth']+data3['product']
data3['GEOGRAPHY0'] = data3.geographic/data3.SUM
data3['GROWTH0'] = data3.growth/data3.SUM
data3['PRODUCT0'] = data3['product']/data3.SUM
data3 = data3[['Id','GEOGRAPHY0','GROWTH0','PRODUCT0']]
data = pandas.merge(left=data,right=data3,how='inner',on=['Id'])
filepath = 'G:/My Drive/Elio/Attention/Data/Results_def_01_for_Elio.txt'
data4 = pandas.read_csv(filepath,sep='\t')
data4 = data4.iloc[1:1008,[0,1,25,26,27]]
data4.columns = ['Year','Id','PRODUCT','GEOGRAPHY','GROWTH']
data4['Year'] = pandas.to_numeric(data4.Year.astype(str).str[0:4])
print(data4)
data = pandas.merge(left=data,right=data4,how='inner',on=['Year','Id'])
print(data.columns)
data

(1232, 33) from 2019_05_25_Results.xlxs
[1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 2005 2006 2007 2008 2009 2010 2011 2012] Years
56 Number of firms
(1539, 3) to get country
[nan 'USA' 'Switzerland' 'Netherlands' 'Germany' 'South Africa' 'Japan'
 'United Kingdom' 'France' 'Australia' 'Ireland' 'Spain' 'Denmark'
 'Sweden']
(1232, 34) correction by Elio of market commonality
4089.0 Total number of Non-RD alliances
7676.0 Total number of RD alliances (hybrid alliances)
      Year  Id  PRODUCT  GEOGRAPHY    GROWTH
1     1995   2     0.00   0.339996  0.660004
2     1995   3      NaN        NaN       NaN
3     1995   4      NaN        NaN       NaN
4     1995   5      NaN        NaN       NaN
5     1995   6      NaN        NaN       NaN
...    ...  ..      ...        ...       ...
1003  2012  52     0.75   0.075352  0.170363
1004  2012  53     0.83   0.000000  0.167799
1005  2012  54     0.00   0.997644  0.002356
1006  2012  55      NaN        NaN       NaN
1007 

,Id,Year,Age,Employees,Total_assets,Net_sales,Human_resources_1,Human_resources_2,Physical_resources_1,Physical_resources_2,...,ROA,ROE,Country,Market_commonality,GEOGRAPHY0,GROWTH0,PRODUCT0,PRODUCT,GEOGRAPHY,GROWTH
0,1,1996,94.0,74289.0,13364.000,14236.000,0.192,5.218,4844.000,0.340,...,0.114,0.243,USA,0.524,0.183858,0.241124,0.575019,0.0,0.784127,0.215873
1,1,1997,95.0,75639.0,13238.000,15070.000,0.199,5.019,5034.000,0.334,...,0.160,0.358,USA,0.237,0.183858,0.241124,0.575019,0.0,0.774649,0.225351
2,1,1998,96.0,73564.0,14153.000,15021.000,0.204,4.897,5566.000,0.371,...,0.083,0.198,USA,0.360,0.183858,0.241124,0.575019,0.0,0.750883,0.249117
3,1,1999,97.0,70549.0,13896.000,15659.000,0.222,4.505,5656.000,0.361,...,0.127,0.280,USA,0.323,0.183858,0.241124,0.575019,0.0,0.793790,0.206210
4,1,2000,98.0,75000.0,14522.000,16724.000,0.223,4.485,5823.000,0.348,...,0.123,0.273,USA,0.240,0.183858,0.241124,0.575019,0.0,0.788899,0.211101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,56,2008,148.0,47426.0,44031.724,22833.908,0.481,2.077,11198.211,0.490,...,0.100,0.230,Denmark,0.555,0.237752,0.190167,0.572081,NaN,NaN,NaN
1003,56,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Denmark,NaN,0.237752,0.190167,0.572081,NaN,NaN,NaN
1004,56,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Denmark,NaN,0.237752,0.190167,0.572081,NaN,NaN,NaN
1005,56,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Denmark,NaN,0.237752,0.190167,0.572081,NaN,NaN,NaN


# Variables

In [23]:
df = data.copy()
df = df.sort_values(by=['Id','Year'],axis=0,ascending=True)

# UNDERPERFORMANCE MAGNITUDE
df['distance'] = df['ROE'] - df.groupby(['Year']).ROE.transform('mean')
df.loc[(df.distance<0),'Underperformance_magnitude']=df['distance']
df.loc[(df.distance>0)|(df.distance==0),'Underperformance_magnitude']=0

# UNDERPERFORMANCE DURATION
df['Underperformance_duration'] = 0
mylist = []
for firm in range(df.Id.nunique()):
  mylist.append(df.loc[df.Id==firm+1,['Id','Year','Underperformance_magnitude','Underperformance_duration']].values.tolist())
m = [[0,0,0]]
for firm in range(df.Id.nunique()):
  for i in mylist[firm]: 
    if i[2]<0:
      m.append([i[0],i[1],m[-1][2]+1])
    else:
      m.append([i[0],i[1],i[3]])
del m[0]
print(len(m))
mydf = pandas.DataFrame(m,columns=['Id','Year','Underperformance_duration'])
df = df.drop('Underperformance_duration',axis=1)
df = pandas.merge(left=df,right=mydf,how='inner',on=['Year','Id'])
print(df.loc[(df.Id==38)|(df.Id==39),['Id','Year','Underperformance_magnitude','Underperformance_duration']])
df = df.sort_values(by=['Id','Year'], axis=0, ascending=True)

# AMBIDEXTROUS ALLIANCES
print(df.Non_RD_alliances.corr(df.RD_alliances),'correlation between RD alliances and non-RD alliances')
df['Ambidextrous_alliances'] = (1/1000)*df.RD_alliances*df.Non_RD_alliances

# ATTENTIONAL FOCUS
df['Attentional_focus'] = df[['PRODUCT','GEOGRAPHY','GROWTH']].apply(lambda x: x.std(skipna=True), axis=1)

# ATTENTIONAL FOCUS SQUARED
df['Attentional_focus_squared'] = (df['Attentional_focus']-df['Attentional_focus'].mean())*(df['Attentional_focus']-df['Attentional_focus'].mean())

# CONTROL VARIABLES
df['distance'] = df['ROE'] - df.groupby(['Year']).ROE.transform('mean')
df.loc[(df.distance>0)|(df.distance==0),'Overperformance_magnitude']=df['distance']
df.loc[(df.distance<0)|(df.distance==0),'Overperformance_magnitude']=0

df['Overperformance_duration'] = 0
mylist = []
for firm in range(df.Id.nunique()):
  mylist.append(df.loc[df.Id==firm+1,['Id','Year','Overperformance_magnitude','Overperformance_duration']].values.tolist())
m = [[0,0,0]]
for firm in range(df.Id.nunique()):
  for i in mylist[firm]: 
    if i[2]>0:
      m.append([i[0],i[1],m[-1][2]+1])
    else:
      m.append([i[0],i[1],i[3]])
del m[0]
print(m)
print(len(m))
mydf = pandas.DataFrame(m,columns=['Id','Year','Overperformance_duration'])
df = df.drop('Overperformance_duration',axis=1)
df = pandas.merge(left=df,right=mydf,how='inner',on=['Year','Id'])
print(df.loc[(df.Id==38)|(df.Id==39),['Id','Year','Overperformance_magnitude','Overperformance_duration']])
df = df.sort_values(by=['Id','Year'], axis=0, ascending=True)

df['Absorbed_slack'] = df['Absorbed_slack']
df['Unabsorbed_slack'] = df['Unabsorbed_slack']
df['Absorptive_capacity'] = df['Absorptive_capacity']
df['Market_commonality'] = df['Market_commonality']
df['Size'] = numpy.log(df.Employees) 

df

1007
     Id  Year  Underperformance_magnitude  Underperformance_duration
665  38  1995                   -0.015244                        5.0
666  38  1996                   -0.098000                        6.0
667  38  1997                    0.000000                        0.0
668  38  1998                   -0.209075                        1.0
669  38  1999                   -0.059585                        2.0
670  38  2000                   -0.124833                        3.0
671  38  2001                   -0.008870                        4.0
672  38  2002                    0.000000                        0.0
673  38  2003                    0.000000                        0.0
674  38  2004                   -0.045434                        1.0
675  38  2005                   -0.000642                        2.0
676  38  2006                   -0.000154                        3.0
677  38  2007                   -0.634098                        4.0
678  38  2008                

,Id,Year,Age,Employees,Total_assets,Net_sales,Human_resources_1,Human_resources_2,Physical_resources_1,Physical_resources_2,...,GROWTH,distance,Underperformance_magnitude,Underperformance_duration,Ambidextrous_alliances,Attentional_focus,Attentional_focus_squared,Overperformance_magnitude,Overperformance_duration,Size
0,1,1996,94.0,74289.0,13364.000,14236.000,0.192,5.218,4844.000,0.340,...,0.215873,0.049000,0.000000,0.0,0.000,0.405045,0.003086,0.049000,1.0,11.215718
1,1,1997,95.0,75639.0,13238.000,15070.000,0.199,5.019,5034.000,0.334,...,0.225351,0.228423,0.000000,0.0,0.000,0.398454,0.003862,0.228423,2.0,11.233727
2,1,1998,96.0,73564.0,14153.000,15021.000,0.204,4.897,5566.000,0.371,...,0.249117,-0.120075,-0.120075,1.0,0.002,0.382460,0.006105,0.000000,0.0,11.205911
3,1,1999,97.0,70549.0,13896.000,15659.000,0.222,4.505,5656.000,0.361,...,0.206210,0.092415,0.000000,0.0,0.000,0.411881,0.002373,0.092415,1.0,11.164063
4,1,2000,98.0,75000.0,14522.000,16724.000,0.223,4.485,5823.000,0.348,...,0.211101,0.115167,0.000000,0.0,0.000,0.408407,0.002724,0.115167,2.0,11.225243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,56,2008,148.0,47426.0,44031.724,22833.908,0.481,2.077,11198.211,0.490,...,NaN,0.037640,0.000000,0.0,0.004,NaN,NaN,0.037640,1.0,10.766926
1003,56,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.012,NaN,NaN,NaN,0.0,NaN
1004,56,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
1005,56,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN


In [24]:
dv = ['Underperformance_magnitude','Underperformance_duration']
iv = ['Ambidextrous_alliances','Attentional_focus','Attentional_focus_squared','Overperformance_magnitude','Overperformance_duration','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size']
mydata = df[dv+iv]
mydata

,Underperformance_magnitude,Underperformance_duration,Ambidextrous_alliances,Attentional_focus,Attentional_focus_squared,Overperformance_magnitude,Overperformance_duration,Absorbed_slack,Unabsorbed_slack,Absorptive_capacity,Market_commonality,Size
0,0.000000,0.0,0.000,0.405045,0.003086,0.049000,1.0,0.256,0.196,0.067,0.524,11.215718
1,0.000000,0.0,0.000,0.398454,0.003862,0.228423,2.0,0.253,0.120,0.066,0.237,11.233727
2,-0.120075,1.0,0.002,0.382460,0.006105,0.000000,0.0,0.252,0.121,0.068,0.360,11.205911
3,0.000000,0.0,0.000,0.411881,0.002373,0.092415,1.0,0.252,0.115,0.066,0.323,11.164063
4,0.000000,0.0,0.000,0.408407,0.002724,0.115167,2.0,0.328,0.635,0.066,0.240,11.225243
...,...,...,...,...,...,...,...,...,...,...,...,...
1002,0.000000,0.0,0.004,NaN,NaN,0.037640,1.0,0.433,2.123,0.148,0.555,10.766926
1003,NaN,0.0,0.012,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1004,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1005,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


# Table 2

In [25]:
ds_mean = mydata.agg(['mean','std','count']).T
corr = mydata.corr()
corr = pandas.concat([ds_mean,corr], axis = 1)
corr = corr.applymap('{:.3f}'.format)
corr['statistics'] = ' '
corr.set_index('statistics',append=True,inplace=True)

def pearsonr_pval(x,y):
  return pearsonr(x,y)[1].round(3)
pval = mydata.corr(method=pearsonr_pval)
pval = pval.applymap('{:.3f}'.format)
pval = pval.applymap(lambda x: '(' + x + ')')
pval['statistics'] = 'p-value'
pval.set_index('statistics',append=True,inplace=True)

corr = pandas.concat([corr,pval],axis=0).sort_index(kind='merge').reindex(dv+iv,axis=0,level=0)
corr.to_excel('Table2.xlsx')
corr

mean     std     count  \
                           statistics                             
Underperformance_magnitude             -0.098   0.215   904.000   
                           p-value        NaN     NaN       NaN   
Underperformance_duration               3.256   5.801  1007.000   
                           p-value        NaN     NaN       NaN   
Ambidextrous_alliances                  0.056   0.128   794.000   
                           p-value        NaN     NaN       NaN   
Attentional_focus                       0.461   0.102   536.000   
                           p-value        NaN     NaN       NaN   
Attentional_focus_squared               0.010   0.012   536.000   
                           p-value        NaN     NaN       NaN   
Overperformance_magnitude               0.098   0.333   904.000   
                           p-value        NaN     NaN       NaN   
Overperformance_duration                1.741   3.548  1007.000   
                           p-value        NaN     NaN       NaN   
Absorbed_slack                          1.966  37.881   679.000   
                           p-value        NaN     NaN       NaN   
Unabsorbed_slack                        1.391   1.988   724.000   
                           p-value        NaN     NaN       NaN   
Absorptive_capacity                     0.233   0.723   899.000   
                           p-value        NaN     NaN       NaN   
Market_commonality                      0.405   0.141   906.000   
                           p-value        NaN     NaN       NaN   
Size                                    9.367   1.765   878.000   
                           p-value        NaN     NaN       NaN   

                                      Underperformance_magnitude  \
                           statistics                              
Underperformance_magnitude                                 1.000   
                           p-value                       (1.000)   
Underperformance_duration                                 -0.154   
                           p-value                       (0.000)   
Ambidextrous_alliances                                     0.109   
                           p-value                       (0.003)   
Attentional_focus                                          0.035   
                           p-value                       (0.422)   
Attentional_focus_squared                                 -0.008   
                           p-value                       (0.850)   
Overperformance_magnitude                                  0.136   
                           p-value                       (0.000)   
Overperformance_duration                                   0.241   
                           p-value                       (0.000)   
Absorbed_slack                                            -0.065   
                           p-value                       (0.094)   
Unabsorbed_slack                                          -0.163   
                           p-value                       (0.000)   
Absorptive_capacity                                       -0.457   
                           p-value                       (0.000)   
Market_commonality                                        -0.083   
                           p-value                       (0.014)   
Size                                                       0.304   
                           p-value                       (0.000)   

                                      Underperformance_duration  \
                           statistics                             
Underperformance_magnitude                               -0.154   
                           p-value                      (0.000)   
Underperformance_duration                                 1.000   
                           p-value                      (1.000)   
Ambidextrous_alliances                                   -0.161   
                           p-value                      (0.0

# Preparing data for regression

https://data.library.virginia.edu/introduction-to-mediation-analysis/

In [29]:
XY = df[dv+iv+['Id','Year']]
XY = XY.dropna()
X = XY[iv+['Year']]
X = X.copy()
X['Intercept'] = 1
X0 = X.drop(['Attentional_focus','Attentional_focus_squared'],axis=1)

In [38]:
model0 = sm.OLS.from_formula('Underperformance_magnitude~Ambidextrous_alliances+Overperformance_magnitude+Overperformance_duration+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+Year',XY)
est_model0 = model0.fit(cov_type='hac-groupsum',cov_kwds={'time':XY.set_index('Year').index,'maxlags':5})
print(est_model0.summary())
model1 = sm.OLS.from_formula('Attentional_focus~Ambidextrous_alliances+Overperformance_magnitude+Overperformance_duration+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+Year',XY)
est_model1 = model1.fit(cov_type='hac-groupsum',cov_kwds={'time':XY.set_index('Year').index,'maxlags':5})
print(est_model1.summary())
model2 = sm.OLS.from_formula('Underperformance_magnitude~Ambidextrous_alliances+Attentional_focus+Attentional_focus_squared+Overperformance_magnitude+Overperformance_duration+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+Year',XY)
est_model2 = model2.fit(cov_type='hac-groupsum',cov_kwds={'time':XY.set_index('Year').index,'maxlags':25})
print(est_model2.summary())
moderators = {'Attentional_focus_squared': 0.02}
mediation = Mediation(model2,model1,'Ambidextrous_alliances','Attentional_focus',moderators=moderators).fit(n_rep=1000)
print(mediation.summary())

                                OLS Regression Results                                
Dep. Variable:     Underperformance_magnitude   R-squared:                       0.371
Model:                                    OLS   Adj. R-squared:                  0.355
Method:                         Least Squares   F-statistic:                     180.2
Date:                        Wed, 05 Jul 2023   Prob (F-statistic):           8.99e-28
Time:                                21:23:38   Log-Likelihood:                 189.85
No. Observations:                         365   AIC:                            -359.7
Df Residuals:                             355   BIC:                            -320.7
Df Model:                                   9                                         
Covariance Type:                 hac-groupsum                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)            0.019807       -0.000787        0.053846    0.060
ACME (treated)            0.019807       -0.000787        0.053846    0.060
ADE (control)             0.030922       -0.087707        0.151457    0.598
ADE (treated)             0.030922       -0.087707        0.151457    0.598
Total effect              0.050729       -0.070037        0.173467    0.402
Prop. mediated (control)  0.213392       -3.607428        4.708865    0.430
Prop. mediated (treated)  0.213392       -3.607428        4.708865    0.430
ACME (average)            0.019807       -0.000787        0.053846    0.060
ADE (average)             0.030922       -0.087707        0.151457    0.598
Prop. mediated (average)  0.213392       -3.607428        4.708865    0.430


In [40]:
model0 = sm.OLS.from_formula('Underperformance_duration~Ambidextrous_alliances+Overperformance_magnitude+Overperformance_duration+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+Year',XY)
est_model0 = model0.fit(cov_type='hac-groupsum',cov_kwds={'time':XY.set_index('Year').index,'maxlags':5})
print(est_model0.summary())
model1 = sm.OLS.from_formula('Attentional_focus~Ambidextrous_alliances+Overperformance_magnitude+Overperformance_duration+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+Year',XY)
est_model1 = model1.fit(cov_type='hac-groupsum',cov_kwds={'time':XY.set_index('Year').index,'maxlags':5})
print(est_model1.summary())
model2 = sm.OLS.from_formula('Underperformance_duration~Ambidextrous_alliances+Attentional_focus+Attentional_focus_squared+Overperformance_magnitude+Overperformance_duration+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+Year',XY)
est_model2 = model2.fit(cov_type='hac-groupsum',cov_kwds={'time':XY.set_index('Year').index,'maxlags':25})
print(est_model2.summary())
moderators = {'Attentional_focus_squared': 0.001}
mediation = Mediation(model2,model1,'Ambidextrous_alliances','Attentional_focus',moderators=moderators).fit(n_rep=1000)
print(mediation.summary())

                                OLS Regression Results                               
Dep. Variable:     Underperformance_duration   R-squared:                       0.422
Model:                                   OLS   Adj. R-squared:                  0.407
Method:                        Least Squares   F-statistic:                     136.1
Date:                       Wed, 05 Jul 2023   Prob (F-statistic):           1.39e-25
Time:                               21:29:00   Log-Likelihood:                -1090.5
No. Observations:                        365   AIC:                             2201.
Df Residuals:                            355   BIC:                             2240.
Df Model:                                  9                                         
Covariance Type:                hac-groupsum                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------

                          Estimate  Lower CI bound  Upper CI bound  P-value
ACME (control)           -1.726050       -3.169012       -0.449206    0.012
ACME (treated)           -1.726050       -3.169012       -0.449206    0.012
ADE (control)            -1.267059       -5.156288        2.664725    0.550
ADE (treated)            -1.267059       -5.156288        2.664725    0.550
Total effect             -2.993109       -7.050872        1.136162    0.142
Prop. mediated (control)  0.500975       -2.243464        4.145667    0.142
Prop. mediated (treated)  0.500975       -2.243464        4.145667    0.142
ACME (average)           -1.726050       -3.169012       -0.449206    0.012
ADE (average)            -1.267059       -5.156288        2.664725    0.550
Prop. mediated (average)  0.500975       -2.243464        4.145667    0.142


In [117]:
0.1*-14

-1.4000000000000001